In [ ]:
from tensorflow.keras.datasets import mnist
import numpy as np

In [ ]:
print("[INFO] loading MNIST dataset...")
(trainX, trainY), (testX, testY) = mnist.load_data()
trainX = trainX / 255.0
testX = testX / 255.0
# add a channel dimension to the images
trainX = np.expand_dims(trainX, axis=-1)
testX = np.expand_dims(testX, axis=-1)

[INFO] loading MNIST dataset...
11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


## Feature Extractor

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

import tensorflow.keras.backend as K

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import MaxPooling2D

def build_siamese_model(inputShape, embeddingDim=48):

	# specify the inputs for the feature extractor network
	inputs = Input(inputShape)

	# define the first set of CONV => RELU => POOL => DROPOUT layers
	x = Conv2D(64, (3, 3), padding="same", activation="relu")(inputs)
	x = MaxPooling2D(pool_size=(2, 2))(x)
	x = Dropout(0.3)(x)

	# second set of CONV => RELU => POOL => DROPOUT layers
	x = Conv2D(64, (3, 3), padding="same", activation="relu")(x)
	x = MaxPooling2D(pool_size=2)(x)
	x = Dropout(0.3)(x)

  # prepare the final outputs
	pooledOutput = GlobalAveragePooling2D()(x)
	outputs = Dense(embeddingDim)(pooledOutput)

	# build the model
	model = Model(inputs, outputs)

	return model

## Siamese Similarity

In [ ]:
def make_pairs(images, labels):

    """
      This function creates positive and negative pairs of images
      from our disney character recognition dataset.
    """
    pairImages = []
    pairLabels = []

    numClasses = len(np.unique(labels))
    idx = [np.where(labels == i)[0] for i in range(0, numClasses)]
    for idxA in range(len(images)):

      currentImage = images[idxA]
      label = labels[idxA]

      idxB = np.random.choice(idx[label])
      posImage = images[idxB]

      # positive pairs have a label 1
      pairImages.append([currentImage, posImage])
      pairLabels.append([1])

      negIdx = np.where(labels != label)[0]
      negImage = images[np.random.choice(negIdx)]

      # negative paris have a label 0
      pairImages.append([currentImage, negImage])
      pairLabels.append([0])

    return (np.array(pairImages), np.array(pairLabels))

In [ ]:
def euclidean_distance(vectors):
	# unpack the vectors into separate lists
	(featsA, featsB) = vectors
	# compute the sum of squared distances between the vectors
	sumSquared = K.sum(K.square(featsA - featsB), axis=1,
		keepdims=True)
	# return the euclidean distance between the vectors
	return K.sqrt(K.maximum(sumSquared, K.epsilon()))

In [ ]:
def plot_training(H, plotPath):
	# construct a plot that plots and saves the training history
	plt.style.use("ggplot")
	plt.figure()
	plt.plot(H.history["loss"], label="train_loss")
	plt.plot(H.history["val_loss"], label="val_loss")
	plt.plot(H.history["accuracy"], label="train_acc")
	plt.plot(H.history["val_accuracy"], label="val_acc")
	plt.title("Training Loss and Accuracy")
	plt.xlabel("Epoch #")
	plt.ylabel("Loss/Accuracy")
	plt.legend(loc="lower left")
	plt.savefig(plotPath)

## Train

In [ ]:
IMG_SHAPE = (28, 28, 3)
BATCH_SIZE = 64
EPOCHS = 100
MODEL_PATH = "siamese_model"
PLOT_PATH = "training_plot.png"

In [ ]:
# prepare the positive and negative pairs
print("[INFO] preparing positive and negative pairs...")
(pairTrain, labelTrain) = make_pairs(trainX, trainY)
(pairTest, labelTest) = make_pairs(testX, testY)

[INFO] preparing positive and negative pairs...


In [ ]:
pairTrain[0].shape, labelTrain[0].shape

((2, 28, 28, 1), (1,))

In [ ]:
pairTrain[1].shape, labelTrain[1].shape

((2, 28, 28, 1), (1,))

In [ ]:
print("[INFO] building siamese network...")
imgA = Input(shape=IMG_SHAPE)
imgB = Input(shape=IMG_SHAPE)

featureExtractor = build_siamese_model(IMG_SHAPE)

featsA = featureExtractor(imgA)
featsB = featureExtractor(imgB)

# finally, construct the siamese network
distance = Lambda(euclidean_distance)([featsA, featsB])
outputs = Dense(1, activation="sigmoid")(distance)
model = Model(inputs=[imgA, imgB], outputs=outputs)

[INFO] building siamese network...


In [ ]:
print("[INFO] compiling model...")
model.compile(loss="binary_crossentropy", optimizer="adam",
	metrics=["accuracy"])

[INFO] compiling model...


In [ ]:
# train the model
print("[INFO] training model...")
history = model.fit(
	[pairTrain[:, 0], pairTrain[:, 1]], labelTrain[:],
	validation_data=([pairTest[:, 0], pairTest[:, 1]], labelTest[:]),
	batch_size=BATCH_SIZE,
	epochs=EPOCHS)

[INFO] training model...
Epoch 1/100
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 94s 50ms/step - accuracy: 0.6242 - loss: 0.6229 - val_accuracy: 0.8490 - val_loss: 0.4334
Epoch 2/100
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 89s 47ms/step - accuracy: 0.8261 - loss: 0.4325 - val_accuracy: 0.8777 - val_loss: 0.3311
Epoch 3/100
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 84s 45ms/step - accuracy: 0.8551 - loss: 0.3566 - val_accuracy: 0.8882 - val_loss: 0.2862
Epoch 4/100
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 83s 44ms/step - accuracy: 0.8663 - loss: 0.3261 - val_accuracy: 0.9060 - val_loss: 0.2481
Epoch 5/100
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 87s 46ms/step - accuracy: 0.8846 - loss: 0.2897 - val_accuracy: 0.9176 - val_loss: 0.2189
Epoch 6/100
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 85s 45ms/step - accuracy: 0.8987 - loss: 0.2592 - val_accuracy: 0.9234 - val_loss: 0.2056
Epoch 7/100
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 83s 44ms/step - accuracy: 0.9065 - loss: 0.2394 - val_accuracy: 0.9364 - val_loss: 0.1744
Epoch 8/100
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 8

In [ ]:
print("[INFO] saving siamese model...")
model.save(MODEL_PATH)

# plot the training history
print("[INFO] plotting training history...")
plot_training(history, PLOT_PATH)